In [1]:
import os
import xarray as xr
import numpy as np
from pathlib import Path
from glob import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import logger
import xesmf as xe
from scipy.ndimage import label

import climtas.nci
from dask.distributed import Client, as_completed
from dask import delayed

In [2]:
client = Client(
    n_workers=3,
    threads_per_worker=1
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 3
Total threads: 3,Total memory: 7.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33915,Workers: 3
Dashboard: /proxy/8787/status,Total threads: 3
Started: Just now,Total memory: 7.00 GiB
Comm: tcp://127.0.0.1:42569,Total threads: 1
Dashboard: /proxy/40831/status,Memory: 2.33 GiB
Nanny: tcp://127.0.0.1:42203,


In [3]:
# Choose bin edges for irradiance values (adjust as needed)
bin_edges = np.linspace(0, 35, 100)  # e.g., 100 bins from 0 to 35
n_bins = len(bin_edges) + 1

In [4]:
def process_file(path, varname='daily_integral_of_surface_global_irradiance'):
    with xr.open_dataset(path) as ds:
        data = ds[varname].load()  # shape: (lat, lon)

    # Digitize data values into bins
    inds = np.digitize(data, bin_edges)

    # Create histogram per (lat, lon)
    hist = np.zeros((n_bins,) + data.shape, dtype=np.int32)

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            bin_idx = inds[i, j]
            hist[bin_idx, i, j] += 1

    return hist

In [ ]:
date = '01-2016'

date_dt = datetime.strptime(date, "%m-%Y")
year = date_dt.strftime("%Y")
month = date_dt.strftime("%m")

##### Himawari Data
if date_dt <= datetime.strptime('2019-03-31', '%Y-%m-%d'):
    version = 'v1.0'
else:
    version = 'v1.1'
directory=Path(f'/g/data/rv74/satellite-products/arc/der/himawari-ahi/solar/p1d/{version}/{year}/{month}')
file_list = sorted(str(p) for p in directory.rglob("*.nc"))

tasks = [delayed(process_file)(path) for path in file_list]

# Parallel computation
futures = client.compute(tasks)
results = client.gather(futures)

ERROR 1: PROJ: proj_create_from_database: Open of /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/share/proj failed


In [ ]:
# Sum over all histograms (axis 0 = bins, axis 1 = lat, axis 2 = lon)
total_hist = np.sum(results, axis=0)  # shape: (bins, lat, lon)

# Get cumulative distribution
cdf = np.cumsum(total_hist, axis=0)
cdf = cdf / cdf[-1]  # normalize

In [ ]:
def percentile_from_cdf(cdf, bin_edges, p):
    idx = np.argmax(cdf >= p, axis=0)
    return bin_edges[idx]

p10 = percentile_from_cdf(cdf, bin_edges, 0.10)